# VacationPy
---
This notebook contains a Python script to analyze weather data of a random set of cities to help with planning future vacations! We use [jupyter-gmaps](https://jupyter-gmaps.readthedocs.io/) and the [Google Places API](https://developers.google.com/places/web-service/intro) to create a heat map that displays the humidity for every city that we gathered weather data for, using the [WeatherPy](https://github.com/anulkar/python-api-challenge/blob/master/WeatherPy/WeatherPy.ipynb) script. We then narrow down the list of cities based on ideal weather conditions for vacation planning purposes and then uses the Places API to find the first hotel for each city located within 5000 meters of the city's geo-coordinates. Lastly, we plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.


 


## Dependencies and Setup

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

## Store Part I Weather Data results into DataFrame

In [16]:
# Path to the csv file containing weather data for random list of cities across the world
cities_csv = "../output_data/cities.csv"

# Read the weather data for all cities into a DataFrame
cities_weather_df = pd.read_csv(cities_csv)

# Print first 5 rows of the DataFrame
cities_weather_df.head()

,City #,City,Country,Date,Latitude,Longitude,Max Temperature,Cloudiness,Humidity,Wind Speed
0,0,Taveta,KE,1580233299,-3.40,37.67,77.00,75,78,5.82
1,1,Butajira,ET,1580233300,8.12,38.37,66.99,64,61,15.48
2,2,Khatanga,RU,1580233301,71.97,102.50,-5.80,81,97,7.99
3,3,Ponta do Sol,PT,1580233301,32.67,-17.10,66.20,20,59,6.93
4,4,Thompson,CA,1580233302,55.74,-97.86,14.00,90,85,4.70


## Humidity Heatmap
* Configure gmaps.
* Create a Heatmap layer that displays the humidity for every city using the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to the gmap.

In [17]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Grab the latitude and longitude locations and humidity data 
locations = cities_weather_df[['Latitude', 'Longitude']]
humidity = cities_weather_df['Humidity'].astype(float)
max_humidity = humidity.max()

# Calculate the center point of all the geo-coordinates for the gmaps figure
center_lat = sum(cities_weather_df['Latitude']) / len(cities_weather_df['Latitude'])
center_long = sum(cities_weather_df['Longitude']) / len(cities_weather_df['Longitude'])
center_coordinates = (center_lat, center_long)

In [18]:
# Plot Heatmap
fig = gmaps.figure(center=center_coordinates, zoom_level=2)

# Create heat layer by using Lat and Lng as location and humidity as the weight
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=float(max_humidity),
                                 point_radius=1)

# Add heatmap layer to map
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame for cities based on ideal weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [19]:
# Narrow down the DataFrame based on ideal weather conditions to return an ideal cities DataFrame
# A max temperature less than or equal to 80 degrees but higher than or equal to 65
# Wind speed less than or equal to 10 mph
# No cloud cover
# Not more than 70% humdity

ideal_weather_conditions_df = cities_weather_df[ (cities_weather_df['Max Temperature'] >= 65) & (cities_weather_df['Max Temperature'] <= 80) & (cities_weather_df['Wind Speed'] <= 10) & (cities_weather_df['Cloudiness'] <= 0)  & (cities_weather_df['Humidity'] <= 70)]

# Drop any rows with null values
ideal_cities_df = ideal_weather_conditions_df.dropna(how='any')

print(f"Here's the DataFrame for {len(ideal_cities_df['City #'])} cities that fit the below ideal weather conditions:")
print("Max Temp >= 65F and <= 80F | Wind Speeds <= 10 | No Cloud Cover | Humidity <= 70%")
ideal_cities_df

Here's the DataFrame for 8 cities that fit the below ideal weather conditions:
Max Temp >= 65F and <= 80F | Wind Speeds <= 10 | No Cloud Cover | Humidity <= 70%


,City #,City,Country,Date,Latitude,Longitude,Max Temperature,Cloudiness,Humidity,Wind Speed
35,35,Guerrero Negro,MX,1580233323,27.98,-114.06,69.12,0,33,3.85
135,135,Abu Dhabi,AE,1580233388,24.47,54.37,66.20,0,63,6.93
144,144,Santa Cruz de Tenerife,ES,1580233396,28.47,-16.25,71.01,0,67,3.36
164,164,Lalmohan,BD,1580233410,22.34,90.74,65.23,0,65,6.20
239,239,Gwadar,PK,1580233450,25.12,62.33,67.91,0,33,2.59
351,351,Pulgaon,IN,1580233534,20.73,78.33,72.16,0,34,3.89
380,380,San Mateo Ixtatan,GT,1580233554,15.83,-91.48,74.23,0,48,5.17
449,449,Magadi,IN,1580233598,12.97,77.23,73.40,0,53,3.36


## Hotel Map
* Using Google Places API, find the first hotel for each city located within 5000 meters of teh city's geo-coordinates.
* Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.





In [20]:
# Set Base URL for the Google Places API
places_api_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Set up lists to hold hotel names, their geo-coordinates, city and country 
hotels_list = []
hotels_lat = []
hotels_lng = []
cities = []
countries = []

# Start of print log as we begin data retrieval using the API
print("Searching for Hotels...")
print(f"-" * 90)

# Iterate over the ideal cities DataFrame to find the first hotel for each city located within 5000 meters of the geo-coordinates.
for index, row in ideal_cities_df.iterrows(): 
    # Setup a dictionary for the parameters to do a 'nearbysearch' of hotels using the Google Places API
    # Parameters include:
    # - the lat/long of city around which to retrieve place information
    # - the radius (in meters) within which to return place results
    # - type of place, i.e. lodging/hotels that we want to search for
    params = { 'location': f"{row['Latitude']}, {row['Longitude']}", 'radius': 5000, 'type': 'lodging', 'key': g_key }
    
    # Execute the Google Place API call with the defined parameters and store the results into a JSON response object
    places_response = requests.get(places_api_url, params=params).json()

    # Try to grab the Hotel Name and it's Geo-Coordinates if available
    try:
        hotel_name = places_response['results'][0]['name']
        hotels_list.append(hotel_name)
        hotels_lat.append(places_response['results'][0]['geometry']['location']['lat'])
        hotels_lng.append(places_response['results'][0]['geometry']['location']['lng'])
        cities.append(row['City'])
        countries.append(row['Country'])
        # Print a log of each city as it's being processed with the city number and city name=
        print(f"Found the hotel '{hotel_name}' within 5000 meters of the city '{row['City']}'")

    # Handle exceptions for any hotels that could not be found within the 5000 meter radius 
    except:
        print(f"Cound not find any hotel within 5000 meters of the city '{row['City']}' | Skipping to the next one...")
        pass

# Print the final log indicating that the search has been completed
print("-" * 90)
print(f"Completed Hotel Search | Below is a list of all {len(hotels_list)} Hotels...")      
print("-" * 90)

# Save the Hotel results to a DataFrame and print it
hotel_df = pd.DataFrame( {'Hotel Name': hotels_list, 'Hotel Lat': hotels_lat, 'Hotel Lng': hotels_lng, 'City': cities, 'Country': countries} )
hotel_df

Searching for Hotels...
------------------------------------------------------------------------------------------
Found the hotel 'Casa Laguna, Bed & Breakfast' within 5000 meters of the city 'Guerrero Negro'
Found the hotel 'Beach Rotana' within 5000 meters of the city 'Abu Dhabi'
Found the hotel 'Hotel Silken' within 5000 meters of the city 'Santa Cruz de Tenerife'
Found the hotel 'Sundorbon courier service' within 5000 meters of the city 'Lalmohan'
Found the hotel 'Sadaf Resort' within 5000 meters of the city 'Gwadar'
Found the hotel 'Shankar Lodge (शंकर लाज)' within 5000 meters of the city 'Pulgaon'
Found the hotel 'HOTEL MAGDALENA' within 5000 meters of the city 'San Mateo Ixtatan'
Found the hotel 'Mindri of Magadi' within 5000 meters of the city 'Magadi'
------------------------------------------------------------------------------------------
Completed Hotel Search | Below is a list of all 8 Hotels...
-----------------------------------------------------------------------------

,Hotel Name,Hotel Lat,Hotel Lng,City,Country
0,"Casa Laguna, Bed & Breakfast",27.959275,-114.055272,Guerrero Negro,MX
1,Beach Rotana,24.494530,54.383850,Abu Dhabi,AE
2,Hotel Silken,28.458413,-16.257442,Santa Cruz de Tenerife,ES
3,Sundorbon courier service,22.340028,90.736658,Lalmohan,BD
4,Sadaf Resort,25.125932,62.319643,Gwadar,PK
5,Shankar Lodge (शंकर लाज),20.723748,78.324614,Pulgaon,IN
6,HOTEL MAGDALENA,15.830499,-91.477092,San Mateo Ixtatan,GT
7,Mindri of Magadi,12.958381,77.228686,Magadi,IN


In [21]:
# Using the template add the hotel markers to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# List of hotel marker locations represents the lat/long coordinates of each hotel
hotel_marker_locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [22]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(hotel_marker_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))